In [1]:
from elasticsearch import Elasticsearch, helpers
from rank_bm25 import BM25Okapi
import pandas as pd
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer
from tqdm import tqdm
import pickle
import numpy as np
from pprint import pprint
import re
from pororo import Pororo

In [2]:
#!service elasticsearch start
#!service elasticsearch stop

In [3]:
!ps -ef | grep elastic

elastic+  5929     1  1 Nov01 ?        00:19:37 /usr/share/elasticsearch/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT --add-opens=java.base/java.io=ALL-UNNAMED -XX:+UseG1GC -Djava.io.tmpdir=/tmp/elasticsearch-5192679610809952812 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=/var/lib/elasticsearch -XX:ErrorFile=/var/log/elasticsearch/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=/var/log/elasticsearch/gc.log:utctime,pid,tags:filecount=32,filesize=64m -Xms31744m -Xmx31744m -XX:MaxDirectMemorySize=16642998272 -XX:InitiatingHeapO

In [4]:
wiki_dataset = pd.read_json('/opt/ml/data/preprocess_wiki.json',orient='index') # 전처리된 위키
wiki_dataset = wiki_dataset.drop_duplicates(['text','title'],ignore_index=True) # 중복 제거

train_dataset = load_from_disk('/opt/ml/data/train_dataset/train').to_pandas()
valid_dataset = load_from_disk('/opt/ml/data/train_dataset/validation').to_pandas()

In [5]:
# ctx_front = []
# ctx_rear = []
# for i in range(len(wiki_dataset)):
#     ctx_len = len(wiki_dataset['text'][i])
#     ctx_front.append(wiki_dataset['text'][i][:ctx_len // 2])
#     ctx_rear.append(wiki_dataset['text'][i][ctx_len // 2:])

# temp1 = wiki_dataset[wiki_dataset.columns]
# temp2 = wiki_dataset[wiki_dataset.columns]

# temp1['text'] = ctx_front
# temp2['text'] = ctx_rear

# total = pd.concat([temp1,temp2],ignore_index=True)
# wiki_dataset = total

In [6]:
# print(ctx_rear[0])
#print(ctx_front[0])
#total['text'][0]


In [7]:
with open('/opt/ml/data/wiki_context_id_pair.bin','rb') as f:
    wiki_context_id_pair = pickle.load(f)

with open('/opt/ml/data/wiki_id_context_pair.bin','rb') as f:
    wiki_id_context_pair = pickle.load(f)

In [8]:
#wiki_corpus = list(set([example_wiki for example_wiki in wiki_dataset['text']]))
train_context = train_dataset['context']
valid_context = valid_dataset['context']
train_query = train_dataset['question']
valid_query = valid_dataset['question']

In [9]:
wiki_corpus = list(wiki_context_id_pair.keys())
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
tokenized_corpus = [tokenizer.tokenize(context) for context in wiki_corpus]
bm25 = BM25Okapi(tokenized_corpus)

Token indices sequence length is longer than the specified maximum sequence length for this model (1131 > 512). Running this sequence through the model will result in indexing errors


In [10]:
INDEX_NAME = "wiki_index"
INDEX_SETTINGS = {
    "settings": {
        "index": {
            "analysis": {
                "analyzer": {
                    "korean": {
                        "type": "custom",
                        "tokenizer": "nori_tokenizer",
                        "filter": ["shingle"],
                    }
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "content": {
                "type": "text",
                "analyzer": "korean",
                "search_analyzer": "korean",
            },
            "title": {
                "type": "text",
                "analyzer": "korean",
                "search_analyzer": "korean",
            },
        }
    },
}
# INDEX_SETTINGS = {
#     "settings": {
#         "analysis": {
#             "analyzer": {
#                 "my_analyzer": {
#                     "type": "custom",
#                     "tokenizer": "nori_tokenizer",
#                     "decompound_mode": "mixed",
#                     "stopwords": "_korean_",
#                     "synonyms": "_korean_",
#                     "filter": [
#                         "lowercase",
#                         "my_shingle_f",
#                         "nori_readingform",
#                         "nori_number",
#                         "cjk_bigram",
#                         "decimal_digit",
#                         "stemmer",
#                         "trim",
#                     ],
#                 }
#             },
#             "filter": {"my_shingle_f": {"type": "shingle"}},
#         },
#         "similarity": {
#             "my_similarity": {
#                 "type": "BM25",
#             }
#         },
#     },
#     "mappings": {
#         "properties": {
#             "title": {
#                 "type": "text",
#                 "analyzer": "my_analyzer",
#                 "similarity": "my_similarity",
#             },
#             "text": {
#                 "type": "text",
#                 "analyzer": "my_analyzer",
#                 "similarity": "my_similarity",
#             },
#             "text_origin": {
#                 "type": "text",
#                 "analyzer": "my_analyzer",
#                 "similarity": "my_similarity",
#             },
#         }
#     },
# }

In [11]:
docs = [
    {
        '_index' : INDEX_NAME,
        '_id' : wiki_dataset.iloc[i]['document_id'],
        'title' : wiki_dataset.iloc[i]['title'],
        'content' : wiki_dataset.iloc[i]['text']
    }
    for i in range(wiki_dataset.shape[0])
]

In [12]:
#!ps -ef | grep elastic

In [22]:
try:
    es.transport.close()
except:
    pass
#config = {'host':'localhost', 'port':2293}
es = Elasticsearch()

In [23]:
es.info()

{'name': '89aa4310917b',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'tWsEBuE7ReuuGa-pZkywtw',
 'version': {'number': '7.15.1',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '83c34f456ae29d60e94d886e455e6a3409bba9ed',
  'build_date': '2021-10-07T21:56:19.031608185Z',
  'build_snapshot': False,
  'lucene_version': '8.9.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [24]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

<ipython-input-24-b31c6028f9a8>:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists(INDEX_NAME):
<ipython-input-24-b31c6028f9a8>:3: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'wiki_index'}

In [25]:
import time
start_time = time.time()
try:
    response = helpers.bulk(es, docs)
    print ("\nRESPONSE:", response)
except Exception as e:
    print("\nERROR:", e)
    
end_time = time.time()

print(end_time-start_time)


RESPONSE: (56044, [])
102.2926025390625


In [27]:
try:
    res = es.search(index=INDEX_NAME, q=train_query[0], size=10)
except:
    res = es.search(index=INDEX_NAME, q=train_query[0], size=10) 

In [18]:
# reader_data
# train_data_with_elastic = {}
# train_data_with_elastic_score = {}

# error_questions = []
# top_k_num = 300

# for i in tqdm(range(len(train_query))):
#     try:
#         res = es.search(index=INDEX_NAME, q=train_query[i], size=top_k_num)
#         hits = res['hits']['hits']
        
#         top_k_list = []
#         top_k_score = []
        
#         for hit in hits:

#             #wiki_id = hit['_id']
#             ctx = hit['_source']['content']
#             score = hit['_score']
            
#             top_k_list.append(wiki_context_id_pair[ctx])
#             top_k_score.append(score)
        
#         train_data_with_elastic[train_query[i]] = top_k_list
#         train_data_with_elastic_score[train_query[i]] = top_k_score
    
#     except Exception as e:
#         error_questions.append([e, train_query[i]])
        
#         top_n_id_text = []
#         top_n_text = bm25.get_top_n(tokenizer.tokenize(train_query[i]), wiki_corpus, n=top_k_num)
        
#         for ctx in top_n_text:
#             #wiki_id = wiki_dataset['document_id'][wiki_dataset['text'] == ctx]
#             top_n_id_text.append(wiki_context_id_pair[ctx])
#         #train_data_with_elastic[train_query[i]] = bm25.get_top_n(tokenizer.tokenize(train_query[i]), wiki_corpus, n=top_k_num)
        
#         train_data_with_elastic[train_query[i]] = top_n_id_text
#         temp_score = bm25.get_scores(tokenizer.tokenize(train_query[i])).tolist()
#         temp_score.sort(reverse=True)
#         train_data_with_elastic_score[train_query[i]] = temp_score[:top_k_num]
    
    

In [28]:
# reader_data
valid_data_with_elastic = {}
valid_data_with_elastic_score = {}
error_questions = []
top_k_num = 300
for i in tqdm(range(len(valid_query))):
    try:
        res = es.search(index=INDEX_NAME, q=valid_query[i], size=top_k_num)
        hits = res['hits']['hits']
        
        top_k_list = []
        top_k_score = []
        
        for hit in hits:

            wiki_id = hit['_id']
            ctx = hit['_source']['content']
            score = hit['_score']
            
            top_k_list.append(wiki_context_id_pair[ctx])
            top_k_score.append(score)
        
        valid_data_with_elastic[valid_query[i]] = top_k_list
        valid_data_with_elastic_score[valid_query[i]] = top_k_score
    
    except Exception as e:
        error_questions.append([e, valid_query[i]])
        
        top_n_id_text = []
        top_n_text = bm25.get_top_n(tokenizer.tokenize(valid_query[i]), wiki_corpus, n=top_k_num)
        
        for ctx in top_n_text:
            #wiki_id = wiki_dataset['document_id'][wiki_dataset['text'] == ctx]
            top_n_id_text.append(wiki_context_id_pair[ctx])
        #train_data_with_elastic[valid_query[i]] = bm25.get_top_n(tokenizer.tokenize(valid_query[i]), wiki_corpus, n=top_k_num)
        
        valid_data_with_elastic[valid_query[i]] = top_n_id_text
        temp_score = bm25.get_scores(tokenizer.tokenize(valid_query[i])).tolist()
        temp_score.sort(reverse=True)
        valid_data_with_elastic_score[valid_query[i]] = temp_score[:top_k_num]

# 반으로 나누었을때
# valid_data_with_elastic = {}
# valid_data_with_elastic_score = {}
# error_questions = []
# top_k_num = 300
# for i in tqdm(range(len(valid_query))):
#     try:

#         res = es.search(index=INDEX_NAME, q=query, size=top_k_num)
#         hits = res['hits']['hits']
        
#         top_k_list = []
#         top_k_score = []
        
#         for hit in hits:

#             wiki_id = hit['_id']
#             ctx = hit['_source']['content']
#             score = hit['_score']
            
#             top_k_list.append(wiki_context_id_pair[wiki_id_context_pair[wiki_id]])
#             top_k_score.append(score)
        
#         valid_data_with_elastic[valid_query[i]] = top_k_list
#         valid_data_with_elastic_score[valid_query[i]] = top_k_score
    
#     except Exception as e:
#         error_questions.append([e, valid_query[i]])
        
#         top_n_id_text = []
#         top_n_text = bm25.get_top_n(tokenizer.tokenize(valid_query[i]), wiki_corpus, n=top_k_num)
        
#         for ctx in top_n_text:
#             #wiki_id = wiki_dataset['document_id'][wiki_dataset['text'] == ctx]
#             top_n_id_text.append(wiki_context_id_pair[ctx])
#         #train_data_with_elastic[valid_query[i]] = bm25.get_top_n(tokenizer.tokenize(valid_query[i]), wiki_corpus, n=top_k_num)
        
#         valid_data_with_elastic[valid_query[i]] = top_n_id_text
#         temp_score = bm25.get_scores(tokenizer.tokenize(valid_query[i])).tolist()
#         temp_score.sort(reverse=True)
#         valid_data_with_elastic_score[valid_query[i]] = temp_score[:top_k_num]

100%|██████████| 240/240 [00:28<00:00,  8.52it/s]


In [29]:
# with open('/opt/ml/data/elastic_train_1000.bin', "wb") as file:
#     pickle.dump(train_data_with_elastic, file)
# with open('/opt/ml/data/elastic_train_score_1000.bin', "wb") as file:
#     pickle.dump(train_data_with_elastic_score, file)
with open('/opt/ml/data/elastic_valid_300.bin', "wb") as file:
    pickle.dump(valid_data_with_elastic, file)
with open('/opt/ml/data/elastic_valid_score_300.bin', "wb") as file:
    pickle.dump(valid_data_with_elastic_score, file)

In [21]:
test_dataset = load_from_disk('/opt/ml/data/test_dataset/validation').to_pandas()
test_query = test_dataset['question']

In [22]:
test_data_with_elastic = {}
test_data_with_elastic_score = {}
error_test_questions = []
top_k_num = 300

for i in tqdm(range(len(test_query))):
    try:
        query = {
            "query": {
                "bool": {
                    "must": [{"match": {"text": valid_query[i]}}],
                    "should": [
                        {
                            "match": {
                                "text": " ".join(
                                    [i[0] for i in ner(valid_query[i]) if i[1] != "O"]
                                )
                            }
                        }
                    ],
                }
            }
        }

        res = es.search(index=INDEX_NAME, q=test_query[i], size=top_k_num)
        hits = res['hits']['hits']
        
        top_k_list = []
        top_k_score = []
        
        for hit in hits:

            wiki_id = hit['_id']
            ctx = hit['_source']['content']
            score = hit['_score']
            
            top_k_list.append(wiki_context_id_pair[ctx])
            top_k_score.append(score)
        
        test_data_with_elastic[test_query[i]] = top_k_list
        test_data_with_elastic_score[test_query[i]] = top_k_score
    
    except Exception as e:
        error_questions.append([e, test_query[i]])
        
        top_n_id_text = []
        top_n_text = bm25.get_top_n(tokenizer.tokenize(test_query[i]), wiki_corpus, n=top_k_num)
        
        for ctx in top_n_text:
            #wiki_id = wiki_dataset['document_id'][wiki_dataset['text'] == ctx]
            top_n_id_text.append(wiki_context_id_pair[ctx])
        #train_data_with_elastic[test_query[i]] = bm25.get_top_n(tokenizer.tokenize(test_query[i]), wiki_corpus, n=top_k_num)
        
        test_data_with_elastic[test_query[i]] = top_n_id_text
        temp_score = bm25.get_scores(tokenizer.tokenize(test_query[i])).tolist()
        temp_score.sort(reverse=True)
        test_data_with_elastic_score[test_query[i]] = temp_score[:top_k_num]

100%|██████████| 600/600 [01:06<00:00,  8.97it/s]


In [20]:
with open('/opt/ml/data/elastic_test_300.bin', "wb") as file:
    pickle.dump(test_data_with_elastic, file)
with open('/opt/ml/data/elastic_test_score_300.bin', "wb") as file:
    pickle.dump(test_data_with_elastic_score, file)

In [26]:
# with open('/opt/ml/data/elastic_train_100.bin', "rb") as file:
#     train_data_with_elastic = pickle.load(file)
with open('/opt/ml/data/elastic_valid_200.bin', "rb") as file:
    valid_data_with_elastic = pickle.load(file)

In [ ]:
INDEX_NAME = "wiki_index"
INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer":"nori_tokenizer",
            "filter": [ "nori_posfilter","nori_readingform","shingle"],
          }
        },
        "filter":{
          "nori_posfilter":{
            "type":"nori_part_of_speech",
            "stoptags":[
                #"E", # 어미
                #"IC", # 감탄사
                #"J", # 조사
                #"MAJ" # 접속부사
                #"MAG", # 일반부사
                #"MM", #관형사
                "NA", #알려지지 않은
                #"NR", #수사
                #"SC",#구분기호
                #"SE",#생략
                #"SF",#마침표, 물음표, 느낌표
                "SH",#한자
                "SL",#외국어
                #"SN",#숫자
                "SP",#띄어쓰기
                "SSC",#닫는괄호
                "SSO",#여는괄호
                "SY",#기타기호
                "UNA",#알수 없는
                "UNKNOWN",#알수없는
                #"VA",#형용사
                #"VCN",#부정지정사
                #"VCP",#긍정 지정사
                "VSV",#알수없는
                #"VV",#동사
                #"VX",#보조동사 또는 형용사
                #"XPN",#접두사
                #"XR",#어근
                #"XSA",#형용사 접미사
                #"XSN",#명사 접미사
                #"XSV"#동사 접미사
            ]
          }
        }
        }
      },
  },
  "mappings": {
      "properties" : {
        "content" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        },
        "title" : {
          "type" : "text",
          "analyzer": "korean",
          "search_analyzer": "korean"
        }
      }
  }
}

In [ ]:
# gq negative sampling

# retrieval_data_with_elastic = {}
# retrieval_data_with_elastic_score = {}
# error_questions = []
# top_k_num = 101
# for i in tqdm(range(len(retrieval_query))):
#     try:

#         res = es.search(index=INDEX_NAME, q=retrieval_query[i], size=top_k_num)
#         hits = res['hits']['hits']

#         top_k_list = []
#         top_k_score = []

#         for hit in hits:

#             wiki_id = hit['_id']
#             #ctx = hit['_source']['content']
#             score = hit['_score']

#             top_k_list.append(int(wiki_id))
#             top_k_score.append(score)

#         if len(top_k_list) == top_k_num:
#             retrieval_data_with_elastic[retrieval_query[i]] = top_k_list
#             retrieval_data_with_elastic_score[retrieval_query[i]] = top_k_score
#         else:
#             print('under_top_k')
#             print(retrieval_query[i])

#     except Exception as e:
#         print('elastic_error')
#         print(retrieval_query[i])

#         error_questions.append([e, retrieval_query[i]])

#         top_n_id_text = []
#         top_n_text = bm25.get_top_n(tokenizer.tokenize(retrieval_query[i]), wiki_corpus, n=top_k_num)
#         if len(top_n_text) == top_k_num:
#             for ctx in top_n_text:

#                 wiki_id = wiki_dataset['document_id'][wiki_dataset['text'] == ctx]
#                 top_n_id_text.append(int(wiki_id))

#             #train_data_with_elastic[retrieval_query[i]] = bm25.get_top_n(tokenizer.tokenize(retrieval_query[i]), wiki_corpus, n=top_k_num)
#             retrieval_data_with_elastic[retrieval_query[i]] = top_n_id_text
#             temp_score = bm25.get_scores(tokenizer.tokenize(retrieval_query[i])).tolist()
#             temp_score.sort(reverse=True)
#             retrieval_data_with_elastic_score[retrieval_query[i]] = temp_score[:top_k_num]